In [1]:
RUNNING_LOCALLY = True

In [2]:
import os

if not RUNNING_LOCALLY:
    os.chdir("/home/yash/")
    print("Current Directory ->", os.getcwd())

    # Ensure that you are working in the right environment
    !echo $CONDA_PREFIX

In [3]:
import gym
import time
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

D:\program\Anaconda\envs\atari\lib\site-packages\flatbuffers\compat.py:19: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  import imp


Num GPUs Available:  0


In [4]:
env = gym.make("ALE/Riverraid-v5")

In [5]:
#num_actions = 18
env.action_space

Discrete(18)

In [6]:
env.observation_space
#we may not need the color? -> turn to grey

Box(0, 255, (210, 160, 3), uint8)

**DQN**

In [7]:
hidden_size = 100
num_actions = 18
q_net = tf.keras.Sequential()
q_net.add(tf.keras.layers.Flatten())
q_net.add(tf.keras.layers.Dense(hidden_size, activation='relu'))
q_net.add(tf.keras.layers.Dense(num_actions, activation='softmax'))
#we can try to add convolutional layers

In [9]:
trainer = tf.keras.optimizers.SGD(learning_rate = 0.1)

**One episode**

In [ ]:
#hyper parameters
num_steps = 500
obs = env.reset()
rTot = 0
gamma = 0.99
e = 0.2


for step in range(num_steps):

    
    with tf.GradientTape() as tape:
        # Compute the Q values and best action for the current state
        q_values = q_net(np.array([obs]))
        action = tf.math.argmax(q_values,axis=1).numpy()[0]
        
        # Epsilon adaptive
        if np.random.rand(1) < e:
            action = env.action_space.sample()

        # apply the action
        nst_obs, reward, done, _ = env.step(action)
        
        
        q_values_nst = q_net(np.array([nst_obs]))
        action_nst = tf.math.argmax(q_values_nst,axis=1).numpy()[0]
        
        td_error = q_values.numpy()
        td_error[0][action] = reward + gamma*action_nst
        loss = tf.reduce_sum(tf.square(td_error-q_values))
        
    grad = tape.gradient(loss,q_net.trainable_variables)
    trainer.apply_gradients(zip(grad,q_net.trainable_variables))
    rTot += reward
    
    obs = nst_obs
    
    if done:
        break
    


    
# Close the env
env.close()

In [ ]:
rTot

**Episodes with n_games**

In [ ]:
#hyperparameters
num_steps = 500
n_games = 15000
E = 1000
rTot = 0
gamma = 0.99
n_games = 15000
obs = env.reset()


for i in range(0, n_games):
    print(f'Running game {i+1}/{n_games}...')
    e = E / (i + E)
    st = game.reset()
    for step in range(num_steps):

    
        with tf.GradientTape() as tape:
            # Compute the Q values and best action for the current state
            q_values = q_net(np.array([obs]))
            action = tf.math.argmax(q_values,axis=1).numpy()[0]

            # Epsilon adaptive
            if np.random.rand(1) < e:
                action = env.action_space.sample()

            # apply the action
            nst_obs, reward, done, _ = env.step(action)


            q_values_nst = q_net(np.array([nst_obs]))
            action_nst = tf.math.argmax(q_values_nst,axis=1).numpy()[0]

            td_error = q_values.numpy()
            td_error[0][action] = reward + gamma*action_nst
            loss = tf.reduce_sum(tf.square(td_error-q_values))

        grad = tape.gradient(loss,q_net.trainable_variables)
        trainer.apply_gradients(zip(grad,q_net.trainable_variables))
        rTot += reward

        obs = nst_obs

        if done:
            break
    
# Close the env
env.close()